# **DATA MINING PROJECT I**



#Importing and loading the Data

## Importing



In [ ]:
# Standard library imports
import os
import sqlite3
import warnings
from datetime import datetime
from collections import Counter
from itertools import product
from math import ceil

# Data manipulation and math
import numpy as np
import pandas as pd
from scipy import stats
import math

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from scipy.cluster.hierarchy import dendrogram

# Preprocessing and dimensionality reduction
from sklearn.base import clone
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Clustering and evaluation metrics
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN, estimate_bandwidth
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, silhouette_samples, pairwise_distances
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# Visualisation settings
warnings.filterwarnings('ignore')
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pydotplus

## Loading Data

We then give access to google Colab of our google drive to be able to import the ```customer_info.csv``` to our project. We then import the file and create a pandas dataframe named - ```df```




In [ ]:
df = pd.read_csv("customer_info.csv")

In [ ]:
df

## Metadata  

- *customer_id* - Identifier of the customer  
- *customer_name* - Name of the customer (contains degree level)  
- *customer_gender* - Gender of the customer  
- *customer_birth_date* - Birth date of the customer  
- *kids_home* - Number of kids at home  
- *teen_home* - Number of teens at home  
- *number_complaints* - Number of complaints formally done by the customer  
- *distinct_stores_visited* - Number of distinct stores visited by the customer  
- *lifetime_spend_groceries* - Total value spent by the customer on groceries  
- *lifetime_spend_electronics* - Total value spent by the customer on electronics  
- *typical_hour* - Typical hour when the customer visits the store  
- *lifetime_spend_vegetables* - Total value spent by the customer on vegetables  
- *lifetime_spend_nonalcohol_drinks* - Total value spent by the customer on non-alcoholic drinks  
- *lifetime_spend_alcohol_drinks* - Total value spent by the customer on alcoholic drinks  
- *lifetime_spend_meat* - Total value spent by the customer on meat  
- *lifetime_spend_fish* - Total value spent by the customer on fish  
- *lifetime_spend_hygiene* - Total value spent by the customer on hygiene  
- *lifetime_spend_videogames* - Total value spent by the customer on video games  
- *lifetime_spend_petfood* - Total value spent by the customer on pet food  
- *lifetime_total_distinct_products* - Number of distinct products bought by the customer (lifetime)  
- *percentage_of_products_bought_promotion* - Percentage of products that were bought with some promotion  
- *year_first_transaction* - Year of the first transaction of the customer  
- *loyalty_card_number* - Number of the customer loyalty card  
- *location_latitude* - Approximate location (<1km range) of the customer's home (Latitude)  
- *location_longitude* - Approximate location (<1km range) of the customer's home (Longitude)  


# 2. Exploratory Data Analysis

This section of the project emcompasses the initial Exploratory Data Analysis

In [ ]:
# Visualising the first 10 rows of the df
df.head(10)

Was checked the different column types in the dataframe

In [ ]:
df.dtypes

The cloumns were rename on our pandas dataframe for clarity and simplification

In [ ]:
df = df.rename(columns={'customer_id': 'id',
    'customer_name': 'name',
    'customer_gender': 'gender',
    'customer_birthdate': 'birthdate',
    'kids_home': 'n_kids',
    'teens_home': 'n_teens',
    'number_complaints': 'n_complaints',
    'distinct_stores_visited': 'n_stores',
    'lifetime_spend_groceries': 'spend_groceries',
    'lifetime_spend_electronics': 'spend_electronics',
    'lifetime_spend_vegetables': 'spend_vegetables',
    'lifetime_spend_nonalcohol_drinks': 'spend_nonalcohol',
    'lifetime_spend_alcohol_drinks': 'spend_alcohol',
    'lifetime_spend_meat': 'spend_meat',
    'lifetime_spend_fish': 'spend_fish',
    'lifetime_spend_hygiene': 'spend_hygiene',
    'lifetime_spend_videogames': 'spend_videogames',
    'lifetime_spend_petfood': 'spend_petfood',
    'lifetime_total_distinct_products': 'n_distinct_products',
    'percentage_of_products_bought_promotion': 'pct_promo',
    'year_first_transaction': 'year_first_transaction',
    'loyalty_card_number': 'loyalty_card'})

## Missing Values

After examining the project dataset, our team has conducted an analysis of the missing values

In [ ]:
df.isna().sum()

In [ ]:
# Checking the percentage of missing values in each column

missing_counts = df.isna().sum()
total_rows = len(df)

missing_percent = (missing_counts / total_rows) * 100
missing_percent = missing_percent.round(2)

missing_percent_str = missing_percent.astype(str) + " %"

missing_percent_str

In [ ]:
df.isna().sum().sum()

The dataset is largely complete, with most features showing missing values below 1%. The only significant outlier is loyalty_card, which lacks data for 18.51% of the entries. To resolve this, k-Nearest Neighbors (kNN) Imputation is utilized. This method fills the gaps by identifying patterns and similarities among the existing data points, ensuring a complete dataset for further analysis.

## Duplicated Values

In [ ]:
df.duplicated().sum()

In [ ]:
# Checking for duplicated values in the column - id
df["id"].duplicated().sum()

In [ ]:
# Checking for duplicated values in the column - loyalty_card
df["loyalty_card"].duplicated().sum()

The column ```loyalty_card``` has 4,058 duplicated values. It is therefore important to assess whether these duplicated loyalty card numbers correspond to the same client appearing multiple times or whether a single loyalty card number is being associated with different clients.

To investigate this, all duplicated loyalty card records were isolated and grouped by loyalty_card. For each loyalty card ID, the number of unique values in the variables name, birthdate, and latitude was computed. These fields act as client identifiers and allow us to detect potential inconsistencies.

If a loyalty card ID shows only one unique value across these attributes, it suggests repeated records of the same client.

Conversely, loyalty card IDs with multiple unique values in any of these fields indicate that the same loyalty card number may be linked to different clients

In [ ]:
#Selecting all the rows which have a duplicated loyaty card
dupes = df[df.duplicated('loyalty_card', keep=False)]

In [ ]:
# Group by loyalty_card ID and count unique values in 'name', 'birthdate', 'latitude' to identify
#data inconsistencies

check = dupes.groupby('loyalty_card')[['name', 'birthdate', 'latitude']].nunique()
print(check)


Taking an example from the above output, the customer with the loyalty card 900084.0 is associated with two different customers. These customers have different names, different birthdates, and different addresses (as indicated by different latitude values).

This is not a simple case of the same customer being entered twice by mistake. This is a case where the same loyalty card number is being registered to different clients.

In [ ]:
# Filter records where all comparison checks are equal to 1
# This means the records match on name, birthdate, and latitude
possible_duplicates = check[
    (check['name'] == 1) &
    (check['birthdate'] == 1) &
    (check['latitude'] == 1)
]

# Step 4: Check how many potential duplicates were found
print(len(possible_duplicates))



The result of 0 confirms that there are no "clean" duplicates in the dataset. This means that whenever a loyalty card ID appears multiple times, it never refers to the same person.

With the results from above, and to make the dataset easier for machine learning models to process we have converterd the column ```loyalty_card``` data into a simple numerical indicator. This new column called ```has_card```  assigns a 1 if the customer has a loyalty card and a 0 if the field is empty. We have implied this from Business Rules

We then remove the original card ID column and ensure the new variable is stored as a float to maintain consistency with other numerical data. This transformation simplifies the data from a complex ID number into a straightforward "Yes/No" feature that represents customer loyalty.

In [ ]:
df["has_card"] = df["loyalty_card"].notna().astype(int)

In [ ]:
df = df.drop(['loyalty_card'], axis=1)


In [ ]:
df['has_card'] = df['has_card'].astype(float)


## Descriptive Statistics

In [ ]:
# Descriptive statistics - Numeric features only
df.describe().T

In [ ]:
# Descriptive statistics - All features
df.describe(include="all").T

From looking to the descriptive statitstics from above we can see 2 clear data quality issues.

First, the ```pct_promotion``` column contains negative values, which is technically impossible as this variable should strictly range between 0 and 1.

Second, the ```year_first_transaction``` columns includes entries for the year 2026. This is logically inconsistent because the dataset was extracted in 2025.


## Numerical/Categorical Split

We have split the columns of our DataFrame into two groups - categorical and numerical - to simplify data handling based on their data type

In [ ]:
categorical = ["gender"]

numerical = ["n_kids", "n_teens", "n_complaints", "n_stores", "spend_groceries", "spend_electronics", "typical_hour", "spend_vegetables", "spend_nonalcohol","spend_alcohol", "spend_meat",
    "spend_fish", "spend_hygiene","spend_videogames", "spend_petfood","n_distinct_products", "pct_promo", "year_first_transaction", "latitude","longitude", "has_card"]

## Incoherences


In this section of the notebook we start by looking at the incoherence of having some rows where the year of ```year_first_transaction``` is 2026 which is not possible since the data was collected in 2025.

This data error was identified in point 2.3 when we analysed the Descriptive statistics


In [ ]:
df["year_first_transaction"].value_counts().sort_index()


In [ ]:
# Checking the percentage of rows that have the year of first transaction as 2026
(df["year_first_transaction"] == 2026).mean() * 100

In [ ]:
# As it was a very small percentage, we have deleted these rows
df = df[df["year_first_transaction"] != 2026]


After assessing that only 0.87% of the rows have a ```year_first_transaction``` of 2026 we have decided to remove them from our dataframe as they account for a very small percentage of the total rows.

After checking the incoherences on the ```year_first_transaction``` column we looked into the incoherences of the ```pct_promo```.

We have identified that there are values of the pct_promo which are below 0 and above 1. From a Business perspective this cannot happen as this value represents the percentage of products, each client has bought with a discount

In [ ]:
mask_invalid = (df["pct_promo"] < 0) | (df["pct_promo"] > 1)

pct_invalid = mask_invalid.mean() * 100
n_invalid = mask_invalid.sum()

pct_invalid, n_invalid



Firstly, we assessed the percentage of entries that have the ```pct_promo```invalid.

3.7% of the entries have an invalid value for the ```pct_promo``` which accounts for 622 rows in our dataframe



In [ ]:
df.loc[mask_invalid, "pct_promo"].describe()


Then we checked how many of these 622 errors were "close" to the boundaries of between -10% and 115%.

In [ ]:
(df.loc[mask_invalid, "pct_promo"]
 .between(-0.10, 1.15)
 .mean())


The number above stated that 76% of the errors are very small, just slightly below 0 or slightly above 1 (-10% and 115% interval).

Since most of the errors were very close to the boundaries instead of deleting the rows all the values that were below 0 in ```pct_promo``` we have set it to 0. All the numbers that were above 1, we have set it to 1. This step was achieved using the code line below


In [ ]:
df["pct_promo"] = df["pct_promo"].clip(0, 1)

## Data Visualization

In [ ]:
# Setting the styles for the graphs
sns.set(style="darkgrid")

### Histograms

To avoid overcrowding the graphics and maintain clarity, we have divided the variables into two groups: spend-related variables and other variables.

In [ ]:
spend =["spend_groceries", "spend_electronics", "spend_vegetables", "spend_nonalcohol","spend_alcohol", "spend_meat",
    "spend_fish", "spend_hygiene","spend_videogames", "spend_petfood"]
others = ["n_kids", "n_teens", "n_complaints", "n_stores", "n_distinct_products", "pct_promo", "year_first_transaction", "latitude","longitude", "typical_hour"]

In [ ]:
# Code to produce the histograms visualisation for the ´spend´ variables
cols = 3
rows = math.ceil(len(spend) / cols)

fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(spend):
    sns.histplot(x=df[feature], ax=axes[i], kde=True, bins=30)
    axes[i].set_title(f"Histogram of {feature}")
    axes[i].tick_params(axis="x", rotation=0)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

A simple analysis from the graphs above allows us to identify that most of the ```spend``` variables histogram plots are right skewed with the exception of the histogram for ```spend_pet_food```which has a normal distribution

In [ ]:
# Code to produce the histograms visualisation for the ´others´ variables

cols = 3
rows = math.ceil(len(others) / cols)

fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(others):
    sns.histplot(x=df[feature], ax=axes[i], kde=True, bins=30)
    axes[i].set_title(f"Histogram of {feature}")
    axes[i].tick_params(axis="x", rotation=0)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

The provided histograms and density plots visualize the distribution of various numerical features, highlighting skewed data in variables such as ```n_kids``` and ```n_stores```, alongside more normally distributed patterns in ```pct_promo``` and geographical coordinates.

### Boxplots


In [ ]:
# Code to create the box plots for the numerical variables

cols = 3
rows = math.ceil(len(numerical) / cols)

fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(numerical):
    sns.boxplot(x=df[feature], ax=axes[i])
    axes[i].set_title(f"Boxplot of {feature}")

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

The numerical variables are analyzed through boxplots to identify distribution patterns and outliers. Significant right-skewness and numerous outliers are identified in all spending categories, such as `spend_groceries` and `spend_meat`, as well as in behavioral counts like `n_kids` and `n_stores`. Conversely, a more balanced and symmetrical spread is observed for `latitude`, `longitude`, and `pct_promo`. Additionally, some isolated outliers are noted in the lower range of `year_first_transaction`, representing earlier customer acquisitions.

### Countplots

In [ ]:
# Countplot for the only categorical category - gender

cols = 3
rows = math.ceil(len(categorical) / cols)

fig, axes = plt.subplots(rows, cols, figsize=(15, 8))
axes = axes.flatten()

for i, feature in enumerate(categorical):
    sns.countplot(x=df[feature].dropna(), ax=axes[i], legend=False)
    axes[i].set_title(f"Countplot of {feature}")
    axes[i].tick_params(axis="x", rotation=45)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

The categorical distribution of the ```gender``` feature is illustrated through a countplot, which indicates a nearly balanced representation between the two classes.

# 3. Data Pre-processing

## Removing Outliers and Log Transformations

As seen in the chapter 2.6.1 there are multiple histograms that are right-skewed.

In [ ]:
df[spend].skew().sort_values(ascending=False)


In [ ]:
spend_rsk =["spend_groceries", "spend_electronics", "spend_vegetables", "spend_nonalcohol","spend_alcohol", "spend_meat", "spend_fish", "spend_hygiene","spend_videogames" ]

In [ ]:
for col in spend_rsk:
    df[col + "_log"] = np.log1p(df[col])


In [ ]:
# Original
df[spend_rsk].skew()

# transformed log
df[[c + "_log" for c in spend_rsk]].skew()



In [ ]:
numerical.append("spend_groceries_log")
numerical.append("spend_electronics_log")
numerical.append("spend_vegetables_log")
numerical.append("spend_nonalcohol_log")
numerical.append("spend_alcohol_log")
numerical.append("spend_meat_log")
numerical.append("spend_fish_log")
numerical.append("spend_hygiene_log")
numerical.append("spend_videogames_log")


To determine the most effective representation for skewed variables, a comparative analysis was performed between original and log-transformed features using **Z-Score** (|z| > 3) and **Interquartile Range (IQR)** methods. The primary objective was to identify which version minimized the presence of statistical outliers to ensure a more robust dataset.


In [ ]:
z_scores = np.abs(stats.zscore(df[numerical], nan_policy="omit"))

z_outliers = {}

for i, col in enumerate(numerical):
    n_outliers = (z_scores[:, i] >= 3).sum()
    z_outliers[col] = n_outliers

In [ ]:
z_outliers_df = (
    pd.DataFrame.from_dict(z_outliers, orient="index", columns=["n_outliers"])
    .assign(pct_outliers=lambda x: x["n_outliers"] / len(df) * 100)
    .sort_index()
)

z_outliers_df


In [ ]:
outliers_iqr = {}
for col in numerical:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_lim = Q1 - 1.5 * IQR
    upper_lim = Q3 + 1.5 * IQR

    n_outliers = ((df[col] < lower_lim) | (df[col] > upper_lim)).sum()

    outliers_iqr[col] = n_outliers

In [ ]:
outliers_iqr_df = (
    pd.DataFrame.from_dict(outliers_iqr, orient="index", columns=["n_outliers"])
    .assign(pct_outliers=lambda x: x["n_outliers"] / len(df) * 100)
    .sort_values("n_outliers", ascending=False)
)

outliers_iqr_df

Now that we've compared each right-skwed variable with and without logarithms, let's decide which one we'll focus on to eliminate outliers.

Based on the results, log transformation proved beneficial for some spending categories:

```spend_electronics```: Outliers were reduced from 2.63% to 1.38% (zscore) and 7.01% to 4.07% (IQR) after the log transformation.

```spend_meat```: The outlier rate dropped from 0.44% to 0.29% (zscore) and 0.96% to 0.37% (IQR) in its log-transformed state.

However, the analysis often revealed an increase in detected anomalies for log-transformed to other variables due to how the transformation compresses the distribution.


In [ ]:
numerical.remove("spend_groceries_log")
numerical.remove("spend_electronics")
numerical.remove("spend_vegetables_log")
numerical.remove("spend_nonalcohol_log")
numerical.remove("spend_alcohol_log")
numerical.remove("spend_meat")
numerical.remove("spend_fish_log")
numerical.remove("spend_hygiene_log")
numerical.remove("spend_videogames_log")

In [ ]:
df=df.drop(columns = ["spend_groceries_log", "spend_electronics", "spend_vegetables_log", "spend_nonalcohol_log", "spend_alcohol_log", "spend_meat", "spend_fish_log", "spend_hygiene_log", "spend_videogames_log"])

In [ ]:
filters1 = (

    (df["spend_groceries"] <= 120000)
    &
    (df["spend_vegetables"] <= 10000)
     &
    (df["spend_nonalcohol"] <= 7000)
    &
    (df["spend_alcohol"] <= 8000)
    &
    (df["spend_fish"] <= 7000)
    &
    (df["spend_hygiene"] <= 3000)
    &
    (df["spend_videogames"] <= 110000)
    &
    (df["spend_petfood"] <= 2500)
     &
    (df["year_first_transaction"] > 1995)
    &
    (df["spend_meat_log"] > 2)
    &
    (df["spend_electronics_log"] > 2)
)




df_1 = df[filters1].copy()

print("Percentage of data removed:", (len(df) - len(df_1)) / len(df) * 100, "%")

In [ ]:
z_scores = np.abs(stats.zscore(df[numerical], nan_policy="omit"))

filters2 = (z_scores < 3).all(axis=1)

df_2 = df[filters2].copy()

print("Percentage of data removed:", (len(df) - len(df_2)) / len(df) * 100, "%")

In [ ]:
Q1 = df[numerical].quantile(0.25)
Q3 = df[numerical].quantile(0.75)
IQR = Q3 - Q1

upper_lim = Q3 + 1.5 * IQR
lower_lim = Q1 - 1.5 * IQR

filters3 = ((df[numerical] >= lower_lim) & (df[numerical] <= upper_lim)).all(axis=1)

df_3 = df[filters3].copy()

print("Percentage of data removed:", (len(df) - len(df_3)) / len(df) * 100, "%")

In [ ]:
df_4 = df[ filters1 |filters2 | filters3].copy()

print("Percentage of data removed:", (len(df) - len(df_4)) / len(df) * 100, "%")

Three distinct filtering approaches are evaluated to clean the dataset:

Manual Filters (filters1): Custom thresholds are established based on visual inspection and domain logic (e.g., spend_groceries <= 120000), resulting in a 2.76% data removal rate.

Z-Score Filter (filters2): A standard $|z| < 3$ threshold is applied, which would remove 19.53% of the data.

IQR Filter (filters3): This more aggressive method would result in the removal of 44.38% of the dataset.

The final dataset is constructed by applying the operation df_4 = df[filters1 | filters2 | filters3]. Under this logic, an observation is only removed if all three methods classify it as an outlier. If even a single method considers the observation valid, it is retained in the dataset.

This approach ensures that only the most extreme and universally agreed-upon anomalies are discarded, resulting in a final removal rate of 2.76% and a clean dataset of 16,365 rows.

In [ ]:
df = df_4.copy()

In [ ]:
df.shape

# Data Normalization

In [ ]:
standardScaler = StandardScaler()
df[numerical] = standardScaler.fit_transform(df[numerical])

In [ ]:
df.head()

# Data Imputation

Following the cleaning process, missing values were addressed using a k-Nearest Neighbors (kNN) Imputation strategy with $k=5$. This technique estimates missing entries by identifying the most similar records in the feature space, ensuring a complete and mathematically consistent dataset for subsequent analysis.

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights="uniform")

In [ ]:
df[numerical] = imputer.fit_transform(df[numerical])

In [ ]:
df[numerical] = standardScaler.inverse_transform(df[numerical])
df.head()

# Feature Engineering

In [ ]:
ano_atual = datetime.now().year

df["birthdate"] = pd.to_datetime(df["birthdate"], errors="coerce")

df["age"] = ano_atual - df["birthdate"].dt.year

df["total_children"] = df["n_kids"] + df["n_teens"]
df["has_children"] = (df["total_children"] > 0).astype(int)

def time_period(h):
    if 6 <= h < 12:
        return "morning"
    elif 12 <= h < 18:
        return "afternoon"
    else:
        return "evening"

df["time_period"] = df["typical_hour"].apply(time_period)
df['loyalty_years'] = ano_atual - df['year_first_transaction']

New variables were derived to capture more nuanced customer behavior:

**age**: Calculated by subtracting the birth year from the current year.

**total_children & has_children**: Created by summing kids and teens and generating a binary indicator for parenthood.

**time_period**: A categorical feature segmenting the typical_hour into morning, afternoon, and evening.

**loyalty_years**: Measures the duration of the customer relationship based on the year of the first transaction.

In [ ]:
numerical.append("age")
numerical.append("loyalty_years")
numerical.append("total_children")
numerical.append("has_children")
categorical.append("time_period")

In [ ]:
df['age'] = imputer.fit_transform(df[['age']])

In [ ]:
df_2=df.copy()

# Incoherences II

Check if there are customers with a birthdate that would make them minors, but who have a year_first_transaction from 10 years ago

In [ ]:
df_2['age_at_first_purchase'] = df_2['year_first_transaction'] - df_2['birthdate'].dt.year

super_childs = df_2[df_2['age_at_first_purchase'] < 10]

print(f"Number of child detected: {len(super_childs)}")
display(super_childs[['name', 'age_at_first_purchase']].head())

Check if minors have purchased alcohol

In [ ]:
underage_alcool = df_2[(df_2['age'] < 18) & (df_2['spend_alcohol'] > 0)]

print(f"Detected {len(underage_alcool)} underage buyers purchasing alcohol.")
display(underage_alcool[['name', 'age', 'spend_alcohol']])

# One-Hot Encoding



In [ ]:
def get_ohc_df(df, feats):
  ohc = OneHotEncoder(sparse_output=False, drop="first")
  ohc_feat = ohc.fit_transform(df[feats])
  ohc_feat_names = ohc.get_feature_names_out()
  ohc_df = pd.DataFrame(ohc_feat, index=df.index, columns=ohc_feat_names)

  df_ohc = pd.concat([df, ohc_df], axis=1)

  return df_ohc, ohc

In [ ]:
df_encoding, ohc = get_ohc_df(df, categorical)


In [ ]:
ohc.get_feature_names_out()

**One-Hot Encoding (OHE)**: Categorical variables like ```gender``` and ```time_period``` were converted into binary features to be compatible with mathematical models.

In [ ]:
df = df_encoding.drop(columns = ["id", "name", "birthdate"])

Variables such as ```id```, ```name```, and ```birthdate``` were dropped as they do not provide predictive value for modeling.

In [ ]:
numerical.append("gender_male")
numerical.append("time_period_evening")
numerical.append("time_period_morning")


# Feature Selection


Pearson Correlation

In [ ]:
fig = plt.figure(figsize=(10,8))

corr = np.round(df[numerical].corr(method="pearson"), decimals=2)

mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,""))

sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True),
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.show()

Dividing features by Perspectives


In [ ]:

## Demographic / Behaviour
demographic_features = ['n_kids', 'n_teens', 'has_children', 'total_children', 'n_complaints', 'n_stores',
    'typical_hour', 'time_period_evening','time_period_morning',
    'year_first_transaction', 'loyalty_years', 'latitude', 'longitude',
    'has_card','age', 'gender_male',  'pct_promo',
    'n_distinct_products'
]



## Preferences
preference_features = [
  'spend_groceries', 'spend_vegetables', 'spend_nonalcohol',
    'spend_alcohol', 'spend_fish', 'spend_hygiene', 'spend_videogames',
    'spend_petfood','spend_meat_log', 'spend_electronics_log'
]
unused_feats= []

df_dem = df[demographic_features].copy()
df_prf = df[preference_features].copy()

In [ ]:
fig = plt.figure(figsize=(10,8))

corr = np.round(df[demographic_features].corr(method="pearson"), decimals=2)

mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,""))

sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True),
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.show()

In [ ]:
df_dem = df_dem.drop(columns=["n_kids", "n_teens" , "year_first_transaction", "latitude", "longitude", "time_period_evening", "time_period_morning", "has_children", "has_card"])

In [ ]:
demographic_features.remove("n_kids")
demographic_features.remove("n_teens")
demographic_features.remove("year_first_transaction")
demographic_features.remove("latitude")
demographic_features.remove("longitude")
demographic_features.remove("time_period_evening")
demographic_features.remove("time_period_morning")
demographic_features.remove("has_children")
demographic_features.remove("has_card")

In [ ]:
numerical.remove("n_kids")
numerical.remove("n_teens")
numerical.remove("year_first_transaction")
numerical.remove("latitude")
numerical.remove("longitude")
numerical.remove("time_period_evening")
numerical.remove("time_period_morning")
numerical.remove("has_children")
numerical.remove("has_card")

In [ ]:
fig = plt.figure(figsize=(10,8))

corr = np.round(df[preference_features].corr(method="pearson"), decimals=2)

mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,""))

sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True),
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.show()

In [ ]:
df_prf = df_prf.drop(columns=["spend_petfood"])

In [ ]:
numerical.remove("spend_petfood")

In [ ]:
preference_features.remove("spend_petfood")

Following the cleaning and engineering phases, features are divided into two distinct perspectives, **Demographic / Behaviour** and **Preferences** to facilitate targeted analysis. High correlations and redundancies are identified through correlation matrices, leading to the removal of several variables to ensure model efficiency.

After dropping redundant features such as `n_kids`, `latitude`, and `has_card`, the **Demographic / Behaviour** division is composed of `n_complaints`, `n_stores`, `typical_hour`, `loyalty_years`, `age`, `gender_male`, `pct_promo`, and `n_distinct_products`.

The **Preferences** division includes consumption metrics such as `spend_groceries`, `spend_vegetables`, `spend_nonalcohol`, `spend_alcohol`, `spend_fish`, `spend_hygiene`, `spend_videogames`, and the log-transformed `spend_meat_log` and `spend_electronics_log`, while `spend_petfood` is excluded.

#Clusters


In [ ]:
df1=df.copy()

In [ ]:
scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])

## Remove noise rows with DBSCAN


This section describes the application of **DBSCAN (Density-Based Spatial Clustering of Applications with Noise)** to identify and remove anomalies that were not captured by previous statistical filters. Unlike standard methods, DBSCAN detects noise based on the spatial density of the data points.


In [ ]:
def split_noise_rows(df, feats, dbs_model):
  ## Splits the dataframe into noise and non-noise
  dbscan_labels = dbs_model.fit_predict(df[feats])

  df_concat = pd.concat([df,
                            pd.Series(dbscan_labels,
                                      name='dbscan_labels',
                                      index=df.index)], axis=1)

  df_noise = df_concat[df_concat['dbscan_labels']==-1].copy()
  df_nonoise = df_concat[df_concat['dbscan_labels']==0].copy()


  return df_noise, df_nonoise, df_concat

In [ ]:
def plot_kdist_graph(df, feats, n_neighbors=36):
  # K-distance graph to find out the right eps value
  ## For each data point, we calculate the average distance
  ## to its n_neighbors

  neigh = NearestNeighbors(n_neighbors=n_neighbors)
  neigh.fit(df[feats])
  distances, _ = neigh.kneighbors(df[feats])

  ## We sort the average distances of the points
  ## And plot this
  distances = np.sort(distances[:, -1])
  plt.ylabel("%d-NN Distance" % n_neighbors)
  plt.xlabel("Points sorted by distance")
  plt.plot(distances)
  plt.show()



In [ ]:
numerical


In [ ]:
plot_kdist_graph(df, numerical)
## The "Knee" or the threshold before we get very large distances (y-axis)
## is at around 3.3

### Determining Hyperparameters

To configure the DBSCAN model, a **k-distance graph** was generated to find the optimal  (epsilon) value.

* **Min Samples ():** The `n_neighbors` parameter was set to **36**, which corresponds to the `min_samples` used in the final model (number of variables multipled by 2). This ensures that the distance plotted represents the exact threshold required for a point to be considered a "core point" in a cluster.

* **Epsilon ():** By analyzing the resulting plot, the "knee" or inflection point—where distances begin to increase sharply—was identified at approximately **3.3**. This value represents the maximum radius used to search for neighbors.

In [ ]:
## Refer to previous notebook for how to tune DBSCAN
## Based on the hyperparameters we found

dbscan = DBSCAN(eps=3.3, min_samples=36, n_jobs=4)

df_noise, df_nonoise, df_combined_noise = split_noise_rows(df, numerical, dbscan)

In [ ]:
## Rename df_nonoise dataframe to df for simplicity
## Remember the df_combined_noise dataframe has both noise and non noise rows

print(df_combined_noise.shape)
print(df_nonoise.shape)
print(df_noise.shape)

df = df_nonoise.copy()



In [ ]:
df_dem = df[demographic_features].copy()
df_prf = df[preference_features].copy()

## Testing different clustering solutions

In [ ]:
# Set up the clusterers
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=42
)

hierarchical = AgglomerativeClustering(
    metric='euclidean'
)


In [ ]:
## Some functions we need
def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst

def get_r2_scores(df, clusterer, min_k=2, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        r2_clust[n] = r2(df, labels)
    return r2_clust


In [ ]:
def get_r2_df(df, feats, kmeans_model, hierar_model):
  # Obtaining the R² scores for each cluster solution

  r2_scores = {}
  r2_scores['kmeans'] = get_r2_scores(df[feats], kmeans_model)

  for linkage in ['complete', 'average', 'single', 'ward']:
      r2_scores[linkage] = get_r2_scores(
          df[feats], hierar_model.set_params(linkage=linkage)
      )

  return pd.DataFrame(r2_scores)

In [ ]:
def plot_r2_scores(r2_scores,
                   plot_title="Demographic Variables:\nR² plot for various clustering methods\n",
                   legend_title="Cluster methods"):
  # Visualizing the R² scores for each cluster solution on demographic variables
  pd.DataFrame(r2_scores).plot.line(figsize=(10,7))

  plt.title(plot_title, fontsize=21)
  plt.legend(title=legend_title, title_fontsize=11)
  plt.xlabel("Number of clusters", fontsize=13)
  plt.ylabel("R² metric", fontsize=13)
  plt.show()

Finding the optimal clusterer on demographic variables

In [ ]:
## The get_r2_df function takes a few minutes to execute

demog_r2_scores = get_r2_df(df, demographic_features, kmeans, hierarchical)
demog_r2_scores

In [ ]:
plot_r2_scores(demog_r2_scores)

Finding the optimal clusterer on preferences variables

In [ ]:
# Set up the clusterers
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=42
)

hierarchical = AgglomerativeClustering(
    metric='euclidean'
)

## The get_r2_df function takes a few minutes to execute

pref_r2_scores = get_r2_df(df, preference_features, kmeans, hierarchical)
pref_r2_scores

In [ ]:
plot_r2_scores(pref_r2_scores,
               plot_title="Preference Variables:\nR² plot for various clustering methods\n",
               )


To evaluate the effectiveness of different clustering algorithms, the R^2 metric is utilized to measure the proportion of variance explained by the resulting clusters. Across both feature perspectives, **K-means** and **Ward’s method** consistently demonstrate the highest performance, outperforming other hierarchical techniques such as complete, average, and single linkage.

### Demographic Perspectives

For the demographic variables, a significant increase in the  value is observed when moving from 2 to **3 clusters**, where the metric reaches approximately **0.23** for K-means and **0.19** for Ward. This number of clusters is selected as it represents an optimal "elbow" point, providing a clear segmentation of customer behavior and age groups without over-complicating the model.

### Preference Perspectives

The preference-based features show even stronger clustering potential, with  values significantly higher than those in the demographic set. Both **K-means** and **Ward** show nearly overlapping performance, with the most substantial gain in variance explained occurring at **4 clusters**. At this level, the models explain roughly **56%** of the total variance, making it the chosen configuration for defining distinct spending profiles.


##K-Means


#### Merging Clusters

Merge using Hierarchical Clustering

To finalize the segmentation, the cluster results from both the Demographic/Behavior and Preference perspectives were merged into a single, unified classification. This integration allows for a multidimensional view of the customer base, linking "who the customers are" with "how they spend."

In [ ]:
perspective_1 = 'pref_labels'
perspective_2 = 'demog_labels'

In [ ]:
# Applying the right clustering (algorithm and number of clusters) for each perspective

kmeans_dem = KMeans(
    n_clusters=3,
    init='k-means++',
    n_init=20,
    random_state=42
)
demog_labels = kmeans_dem.fit_predict(df_dem)

df['demog_labels'] = demog_labels

In [ ]:
kmeans_pref = KMeans(
    n_clusters=4,
    init='k-means++',
    n_init=20,
    random_state=42
)
pref_labels = kmeans_pref.fit_predict(df_prf)

df['pref_labels'] = pref_labels


In [ ]:
# Count label frequencies (contigency table)

def make_contingency_table(df, label1, label2):
  df_ = df.groupby([label1, label2])\
            .size()\
            .to_frame()\
            .reset_index()\
            .pivot(index=label2, columns=label1)
  df_.columns = df_.columns.droplevel()
  return df_



In [ ]:
make_contingency_table(df, perspective_1, perspective_2)


In [ ]:
def get_mean_bylabel(df, feats, label_name):
  # Characterizing the clusters
  return df[feats+[label_name]].groupby(label_name).mean()


In [ ]:
# Centroids of the concatenated cluster labels
df_hc_centroids = df.groupby([perspective_1, perspective_2])[numerical].mean()
df_hc_centroids

In [ ]:
## Map combinations of the two labels to their merged cluster label

def hc_merge_mapper(df, label1, label2, feats, merged_label, n_clusters=1):
  df_ = df.copy()

  # Centroids of the concatenated cluster labels
  df_centroids = df_.groupby([label1, label2])[feats].mean()

  # Re-running the Hierarchical clustering based on the correct number of clusters
  hclust = AgglomerativeClustering(
      linkage='ward',
      metric='euclidean',
      n_clusters=n_clusters
  )
  hclust_labels = hclust.fit_predict(df_centroids)
  df_centroids[merged_label] = hclust_labels

  cluster_mapper = df_centroids[merged_label].to_dict()

  # Mapping the clusters on the centroids to the observations
  df_[merged_label] = df_.apply(
      lambda row: cluster_mapper[
          (row[label1], row[label2])
      ], axis=1
  )

  return df_, df_centroids


In [ ]:
df_hc_merged, df_hc_centroids = hc_merge_mapper(df, perspective_1, perspective_2, numerical, 'hc_merged_labels', 4)


In [ ]:
df_hc_merged['hc_merged_labels'].value_counts()

**Final Cluster Selection**

Result: A final solution of 4 clusters was selected for the merged dataset.

Justification: Although a 5-cluster configuration was evaluated, it was ultimately discarded because the fifth segment was found to contain too few observations. Maintaining only 4 clusters ensures that each group is sufficiently large to be statistically significant and actionable for business strategies.

In [ ]:
## Get final DF
df = df_hc_merged.copy()

### Cluster Profiles

In [ ]:
def cluster_profiles(df,
                     label_columns,
                     figsize,
                     compar_titles=None,
                     colors='Set1'):
    """
    Pass df with labels columns of one or multiple clustering labels.
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)

    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2,
                             figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):

        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)

        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]

        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label,
                                         color=sns.color_palette(palette=colors), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1],
                    palette=sns.color_palette(palette=colors))

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy')
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
        ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)

    plt.subplots_adjust(hspace=1.5, top=0.90)
    #plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:
# Reminder:
# perspective_1 = 'pref_labels'
# perspective_2 = 'demog_labels'

merged_label_name = 'hc_merged_labels'

In [ ]:
labels_list = [perspective_2, perspective_1, merged_label_name]

In [ ]:
# Profilling each cluster (only merged)
merged_label_list = [merged_label_name]
sns.set(style="white")
cluster_profiles(
    df = df[numerical + merged_label_list],
    label_columns = merged_label_list,
    figsize = (27, 8),
    compar_titles = ["Merged clusters profiling"],
    colors='Set2'
)


In [ ]:
# Profilling each cluster (product, behavior, merged)
sns.set(style="white")
cluster_profiles(
    df = df[numerical + labels_list],
    label_columns = labels_list,
    figsize = (28, 13),
    compar_titles = ["Demographic clustering", "Preference clustering", "Merged clusters"],
    colors='Set2'
)


### Visualize profiles using heatmaps

In [ ]:
def cluster_heatmaps(df,
                     label_columns,
                     figsize=(20,20),
                     compar_titles=None,
                     heat_colors='RdYlBu',
                     bar_colors='Set2'):
    """
    Pass df with labels columns of one or multiple clustering labels.
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)

    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2,
                             figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):

        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)

        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]


        # Setting Data
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(counts.shape[0])]

        sns.heatmap(centroids.drop(columns=label),
              square=False, cmap=heat_colors,
              ax=ax[0],
              )

        ax[0].set_title("Cluster Means Heatmap - {} Clusters".format(counts.shape[0]), fontsize=18)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
        ax[0].set_yticklabels(cluster_labels, rotation=0)
        ax[1].annotate(text=titl, xy=(-0.3,1.15),
                       xycoords='axes fraction',
                       fontsize=18, fontweight = 'heavy')


        sns.barplot(y=label, x="counts", data=counts, ax=ax[1], orient='h', palette=bar_colors)
        ax[1].set_yticklabels(cluster_labels)
        ax[1].set_title("Cluster Sizes - {} Clusters".format(counts.shape[0]), fontsize=18)
        ax[1].set_ylabel("")

    plt.subplots_adjust(hspace=1, top=0.90)
    plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:

# Profilling each cluster (only merged)
merged_label_list = [merged_label_name]
sns.set(style="white")
cluster_heatmaps(
    df = df[numerical  + merged_label_list],
    label_columns = merged_label_list,
    figsize = (27, 8),
    compar_titles = ["Merged clusters profiling"],
    bar_colors='Set2'
)

sns.set()


In [ ]:
# Profilling each cluster (product, behavior, merged)
sns.set(style="whitegrid")
cluster_heatmaps(
    df = df[numerical  + labels_list],
    label_columns = labels_list,
    figsize = (28, 13),
    compar_titles = ["Demographic clustering", "Preference clustering", "Merged clusters"],
)


### Tabular cluster characterization

### Recover original values

Remember, we scaled our data during preprocessing

In [ ]:
## Check the variable name of the scaler you chose to use

scaler

In [ ]:
scaled_feature_names = scaler.get_feature_names_out()
scaled_feature_names

In [ ]:
## Get the inverse of the transformed values
pd.DataFrame(scaler.inverse_transform(df[scaled_feature_names]),
             columns=scaled_feature_names)

In [ ]:
## Get the inverse of the transformed values
pd.DataFrame(scaler.inverse_transform(df[scaled_feature_names]),
             columns=scaled_feature_names)

In [ ]:
## Put this into a DF

df_unscaled = df.copy()

df_unscaled[scaled_feature_names] = pd.DataFrame(scaler.inverse_transform(df[scaled_feature_names]),
                           index=df.index)

df_unscaled.head()

In [ ]:
cluster_means = get_mean_bylabel(df_unscaled,
                                 numerical,
                                 merged_label_name)

cluster_means.style.format(precision=2).background_gradient(axis=0)

In [ ]:
## Do the same with pref_labels and demog_labels instead of merged_labels


In [ ]:
cluster_means_1 = get_mean_bylabel(df_unscaled,
                                 numerical,
                                 perspective_1)
cluster_means_1
cluster_means_1.style.format(precision=2).background_gradient(axis=0)

In [ ]:
cluster_means_2 = get_mean_bylabel(df_unscaled,
                                 numerical,
                                 perspective_2)

cluster_means_2.round(2)
cluster_means_2.style.format(precision=2).background_gradient(axis=0)


### Non-metric features profiling

In [ ]:
print(categorical)

In [ ]:
df["time_period"] = df["time_period"].astype("string")
df["gender"] = df["gender"].astype("string")

In [ ]:
## Characteristics considering merged labels segmentation (non metric)
## MODE of each feature for each cluster

df.groupby([merged_label_name])[categorical].agg(pd.Series.mode)

In [ ]:
# Defining the variables based on the image
non_metric_features = ['gender', 'time_period']
merged_label_name = 'hc_merged_labels'


In [ ]:
def plot_stacked_bars(df, features, cluster_col):
    pastel_colors = ['#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#F9F7CF']

    for feat in features:
        cross_tab = pd.crosstab(df[cluster_col], df[feat], normalize='index') * 100

        fig, ax = plt.subplots(figsize=(10, 6), facecolor='white')


        cross_tab.plot(kind='bar', stacked=True, ax=ax, color=pastel_colors, edgecolor='white')


        for p in ax.patches:
            width, height = p.get_width(), p.get_height()
            if height > 5:
                ax.annotate(f'{height:.1f}%', (p.get_x() + width/2, p.get_y() + height/2),
                            ha='center', va='center', fontsize=9, fontweight='bold', color='#444444')


        ax.set_facecolor('white')
        ax.grid(False)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        # ----------------------------------

        plt.title(f"{feat.upper()} DISTRIBUTION", fontsize=14, pad=20, fontweight='bold')
        plt.legend(title=feat, bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False)
        plt.ylabel("Percentage (%)")
        plt.xlabel("Cluster Number")
        plt.xticks(rotation=0)

        plt.tight_layout()
        plt.show()

# Exemplo de uso:
plot_stacked_bars(df, ['gender', 'time_period'], 'hc_merged_labels')

In [ ]:
def plot_grouped_bars(df, features, cluster_col):

    custom_pastel = ['#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#F9F7CF']

    for feat in features:

        sns.set_style("white")
        plt.figure(figsize=(12, 6), facecolor='white')


        ax = sns.countplot(
            data=df,
            x=cluster_col,
            hue=feat,
            palette=custom_pastel
        )


        for p in ax.patches:
            if p.get_height() > 0:
                ax.annotate(f'{int(p.get_height())}',
                            (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha = 'center', va = 'center',
                            xytext = (0, 9),
                            textcoords = 'offset points',
                            fontsize=10,
                            fontweight='bold',
                            color='#555555')

        plt.title(f"{feat.upper()} COUNT", fontsize=15, pad=20, fontweight='bold')
        plt.xlabel("Cluster Number", fontsize=12)
        plt.ylabel("Quantity", fontsize=12)

        plt.legend(title=feat, bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False)

        sns.despine()


        ax.set_facecolor('white')

        plt.tight_layout()
        plt.show()


plot_grouped_bars(df, ['gender', 'time_period'], 'hc_merged_labels')

### Cluster Visualization using TSNE

In [ ]:
def plot_tsne(df, feats, label,
              cmap='tab10',
              title="t-SNE Visualization of K-Means Clustering Solution"):

  two_dim = TSNE(random_state=42).fit_transform(df[feats])
  two_dim_df = pd.DataFrame(two_dim, index=df.index)
  two_dim_df[label] = df[label]


  fig, ax= plt.subplots(figsize=(10,10))
  scatter = ax.scatter(x = two_dim_df[0],
                      y=two_dim_df[1],
                      c=two_dim_df[label],
                      s=5,
                      cmap=cmap
                      )
  ax.set_xlabel("")
  ax.set_ylabel("")
  ax.set_xticks([])
  ax.set_yticks([])

  legend1 = ax.legend(*scatter.legend_elements(),
                      loc="best", title="Cluster Labels")
  ax.add_artist(legend1)

  plt.title(title)
  plt.show()

In [ ]:
plot_tsne(df, numerical, merged_label_name)

The final clustering solution is visualized using t-SNE (t-Distributed Stochastic Neighbor Embedding) to validate the separation of the groups in a two-dimensional space. After merging the results from the Demographic (3 clusters) and Preference (4 clusters) perspectives using Hierarchical Clustering (Ward's Method), a final configuration of 4 clusters is retained.



### Decision Tree

To interpret the final clustering results and ensure the entire dataset is actionable, a **Decision Tree Classifier** is implemented. This stage serves two primary purposes: providing model interpretability and re-integrating the "noise" observations removed during the density-based filtering phase.

---

### **1. Model Training and Performance**

A Decision Tree with a `max_depth` of 4 is fitted using the numerical features to predict the merged cluster labels. The model achieves an estimated accuracy of **95.10%** on the test set. This high performance indicates that the clusters are well-defined and clearly separable based on the selected features, confirming the robustness of the previous segmentation steps.

### **2. Feature Importance and Interpretability**

The importance of each feature in defining the cluster boundaries is assessed to understand the drivers of customer segmentation:

* **Primary Drivers:** Spending habits are the most significant predictors, with **`spend_groceries`** (**37.47%**), **`spend_meat_log`** (**36.02%**), and **`spend_videogames`** (**19.33%**) contributing the most to the model's decisions.
* **Secondary Drivers:** Features like `spend_electronics_log` and `spend_vegetables` have minor roles, while most demographic variables (e.g., `age`, `gender`, `total_children`) show **zero importance** in this specific classification.
This suggests that while demographic data helped group the users initially, their **actual spending behavior** is what truly distinguishes one segment from another.

### **3. Re-integrating Noise Observations**

The trained Decision Tree is used to predict cluster labels for the **noise rows** that were previously removed by the DBSCAN algorithm. By applying the logic learned from the "clean" data, these outliers are assigned to the most statistically similar cluster. This approach allows for the inclusion of the entire population in final business reports or targeted marketing strategies without compromising the initial cluster definitions.


In [ ]:
# Preparing the data
X = df[numerical]
y = df[merged_label_name]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fitting the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=4)
dt.fit(X_train, y_train)
print("It is estimated that on average, we are able to predict {0:.2f}% of the customers correctly".format(dt.score(X_test, y_test)*100))


In [ ]:
# Assessing feature importance
pd.Series(dt.feature_importances_, index=X_train.columns).sort_values(ascending=False)


In [ ]:
## Remember our noise rows that we removed with DBSCAN?
## Predicting the cluster labels of the outliers

df_noise[merged_label_name] = dt.predict(df_noise[numerical])
df_noise[numerical + [merged_label_name]].head()

In [ ]:
# Visualizing the decision tree
num_of_clusters = len(df[merged_label_name].unique())
cluster_names = ["Cluster {}".format(i) for i in range(num_of_clusters)]
dot_data = export_graphviz(dt, out_file=None,
                           feature_names=X.columns.to_list(),
                           filled=True,
                           rounded=True,
                           class_names=cluster_names,
                           special_characters=True)


In [ ]:

graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('cluster_decision_tree.png')


## Hierarchical Clustering

In [ ]:
scaler = StandardScaler()
df1[numerical] = scaler.fit_transform(df1[numerical])

### Remove noise rows with DBSCAN


In [ ]:
def split_noise_rows(df, feats, dbs_model):
  ## Splits the dataframe into noise and non-noise
  dbscan_labels = dbs_model.fit_predict(df1[feats])

  df_concat = pd.concat([df1,
                            pd.Series(dbscan_labels,
                                      name='dbscan_labels',
                                      index=df.index)], axis=1)

  df_noise = df_concat[df_concat['dbscan_labels']==-1].copy()
  df_nonoise = df_concat[df_concat['dbscan_labels']==0].copy()


  return df_noise, df_nonoise, df_concat

In [ ]:
## Refer to previous notebook for how to tune DBSCAN
## Based on the hyperparameters we found

dbscan = DBSCAN(eps=3.3, min_samples=36, n_jobs=4)

df_noise, df_nonoise, df_combined_noise = split_noise_rows(df1, numerical, dbscan)

In [ ]:
## Rename df_nonoise dataframe to df for simplicity
## Remember the df_combined_noise dataframe has both noise and non noise rows

print(df_combined_noise.shape)
print(df_nonoise.shape)
print(df_noise.shape)

df1 = df_nonoise.copy()



In [ ]:
df_dem = df1[demographic_features].copy()
df_prf = df1[preference_features].copy()

In [ ]:
perspective_1 = 'pref_labels'
perspective_2 = 'demog_labels'

In [ ]:
# Performing HC
hclust_dem = AgglomerativeClustering(linkage='ward',
                                 metric='euclidean',
                                 n_clusters=3)
demog_labels = hclust_dem.fit_predict(df_dem)
df1['demog_labels'] = demog_labels



In [ ]:
# Performing HC
hclust_pref = AgglomerativeClustering(linkage='ward',
                                 metric='euclidean',
                                 n_clusters=4)
pref_labels = hclust_pref.fit_predict(df_prf)
df1['pref_labels'] = pref_labels


In [ ]:
# Count label frequencies (contigency table)

def make_contingency_table(df, label1, label2):
  df_ = df.groupby([label1, label2])\
            .size()\
            .to_frame()\
            .reset_index()\
            .pivot(index=label2, columns=label1)
  df_.columns = df_.columns.droplevel()
  return df_



In [ ]:
make_contingency_table(df1, perspective_1, perspective_2)


In [ ]:
def get_mean_bylabel(df1, feats, label_name):
  # Characterizing the clusters
  return df1[feats+[label_name]].groupby(label_name).mean()


In [ ]:
# Centroids of the concatenated cluster labels
df_hc_centroids = df1.groupby([perspective_1, perspective_2])[numerical].mean()
df_hc_centroids

In [ ]:
## Map combinations of the two labels to their merged cluster label

def hc_merge_mapper(df, label1, label2, feats, merged_label, n_clusters=1):
  df_ = df.copy()

  # Centroids of the concatenated cluster labels
  df_centroids = df_.groupby([label1, label2])[feats].mean()

  # Re-running the Hierarchical clustering based on the correct number of clusters
  hclust = AgglomerativeClustering(
      linkage='ward',
      metric='euclidean',
      n_clusters=n_clusters
  )
  hclust_labels = hclust.fit_predict(df_centroids)
  df_centroids[merged_label] = hclust_labels

  cluster_mapper = df_centroids[merged_label].to_dict()

  # Mapping the clusters on the centroids to the observations
  df_[merged_label] = df_.apply(
      lambda row: cluster_mapper[
          (row[label1], row[label2])
      ], axis=1
  )

  return df_, df_centroids


In [ ]:
df_hc_merged, df_hc_centroids = hc_merge_mapper(df1, perspective_1, perspective_2, numerical, 'hc_merged_labels', 4
                                          )


In [ ]:
df_hc_merged['hc_merged_labels'].value_counts()

In [ ]:
## Get final DF
df1 = df_hc_merged.copy()

### Cluster Profiles

In [ ]:
def cluster_profiles(df1,
                     label_columns,
                     figsize,
                     compar_titles=None,
                     colors='Set1'):
    """
    Pass df with labels columns of one or multiple clustering labels.
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)

    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2,
                             figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):

        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df1.drop(drop_cols, axis=1)

        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]

        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label,
                                         color=sns.color_palette(palette=colors), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1],
                    palette=sns.color_palette(palette=colors))

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy')
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
        ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)

    plt.subplots_adjust(hspace=2, top=0.90)
    plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:
# Reminder:
# perspective_1 = 'pref_labels'
# perspective_2 = 'demog_labels'

merged_label_name = 'hc_merged_labels'

In [ ]:
labels_list = [perspective_2, perspective_1, merged_label_name]

In [ ]:
# Profilling each cluster (only merged)
merged_label_list = [merged_label_name]
sns.set(style="white")
cluster_profiles(
    df1 = df1[numerical + merged_label_list],
    label_columns = merged_label_list,
    figsize = (27, 8),
    compar_titles = ["Merged clusters profiling"],
    colors='Set2'
)


In [ ]:
# Profilling each cluster (product, behavior, merged)
sns.set(style="white")
cluster_profiles(
    df1 = df1[numerical + labels_list],
    label_columns = labels_list,
    figsize = (28, 13),
    compar_titles = ["Demographic clustering", "Preference clustering", "Merged clusters"],
    colors='Set2'
)


### Visualize profiles using heatmaps

In [ ]:
def cluster_heatmaps(df1,
                     label_columns,
                     figsize=(20,20),
                     compar_titles=None,
                     heat_colors='RdYlBu',
                     bar_colors='Set2'):
    """
    Pass df with labels columns of one or multiple clustering labels.
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)

    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2,
                             figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):

        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df1.drop(drop_cols, axis=1)

        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]


        # Setting Data
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(counts.shape[0])]

        sns.heatmap(centroids.drop(columns=label),
              square=False, cmap=heat_colors,
              ax=ax[0],
              )

        ax[0].set_title("Cluster Means Heatmap - {} Clusters".format(counts.shape[0]), fontsize=18)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
        ax[0].set_yticklabels(cluster_labels, rotation=0)
        ax[1].annotate(text=titl, xy=(-0.3,1.15),
                       xycoords='axes fraction',
                       fontsize=18, fontweight = 'heavy')


        sns.barplot(y=label, x="counts", data=counts, ax=ax[1], orient='h', palette=bar_colors)
        ax[1].set_yticklabels(cluster_labels)
        ax[1].set_title("Cluster Sizes - {} Clusters".format(counts.shape[0]), fontsize=18)
        ax[1].set_ylabel("")

    plt.subplots_adjust(hspace=2, top=0.90)
    plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:

# Profilling each cluster (only merged)
merged_label_list = [merged_label_name]
sns.set(style="white")
cluster_heatmaps(
    df1 = df1[numerical  + merged_label_list],
    label_columns = merged_label_list,
    figsize = (27, 8),
    compar_titles = ["Merged clusters profiling"],
    bar_colors='Set2'
)

sns.set()


In [ ]:
# Profilling each cluster (product, behavior, merged)
sns.set(style="whitegrid")
cluster_heatmaps(
    df1 = df1[numerical  + labels_list],
    label_columns = labels_list,
    figsize = (28, 13),
    compar_titles = ["Demographic clustering", "Preference clustering", "Merged clusters"],
)


### Tabular cluster characterization

### Recover original values

Remember, we scaled our data during preprocessing

In [ ]:
## Check the variable name of the scaler you chose to use

scaler

In [ ]:
scaled_feature_names = scaler.get_feature_names_out()
scaled_feature_names

In [ ]:
## Get the inverse of the transformed values
pd.DataFrame(scaler.inverse_transform(df1[scaled_feature_names]),
             columns=scaled_feature_names)

In [ ]:
## Get the inverse of the transformed values
pd.DataFrame(scaler.inverse_transform(df1[scaled_feature_names]),
             columns=scaled_feature_names)

In [ ]:
## Put this into a DF

df_unscaled = df1.copy()

df_unscaled[scaled_feature_names] = pd.DataFrame(scaler.inverse_transform(df[scaled_feature_names]),
                           index=df.index)

df_unscaled.head()

In [ ]:
cluster_means = get_mean_bylabel(df_unscaled,
                                 numerical,
                                 merged_label_name)

cluster_means.style.format(precision=2).background_gradient(axis=0)

In [ ]:
## Do the same with pref_labels and demog_labels instead of merged_labels


In [ ]:
cluster_means_1 = get_mean_bylabel(df_unscaled,
                                 numerical,
                                 perspective_1)
cluster_means_1
cluster_means_1.style.format(precision=2).background_gradient(axis=0)

In [ ]:
cluster_means_2 = get_mean_bylabel(df_unscaled,
                                 numerical,
                                 perspective_2)

cluster_means_2.round(2)
cluster_means_2.style.format(precision=2).background_gradient(axis=0)


### Non-metric features profiling

In [ ]:
print(categorical)

In [ ]:
## Characteristics considering merged labels segmentation (non metric)
## MODE of each feature for each cluster

df1.groupby([merged_label_name])[categorical].agg(pd.Series.mode)

In [ ]:
# Defining the variables based on your image
non_metric_features = ['gender', 'time_period']
merged_label_name = 'hc_merged_labels'


In [ ]:

def plot_stacked_bars(df, features, cluster_col):
    # Defining a custom pastel palette
    pastel_colors = ['#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#F9F7CF']

    for feat in features:
        cross_tab = pd.crosstab(df[cluster_col], df[feat], normalize='index') * 100

        # Creating the figure and defining the white background
        fig, ax = plt.subplots(figsize=(10, 6), facecolor='white')

        cross_tab.plot(kind='bar', stacked=True, ax=ax, color=pastel_colors, edgecolor='white')

        # Add the percentage labels
        for p in ax.patches:
            width, height = p.get_width(), p.get_height()
            if height > 5:
                ax.annotate(f'{height:.1f}%', (p.get_x() + width/2, p.get_y() + height/2),
                            ha='center', va='center', fontsize=9, fontweight='bold', color='#444444')

        # --- BACKGROUND AND AESTHETIC ADJUSTMENTS ---
        ax.set_facecolor('white')           # Background of the white chart area
        ax.grid(False)                      # remove the grid lines
        ax.spines['top'].set_visible(False) # Remove top border
        ax.spines['right'].set_visible(False) # Remove right border
        # ----------------------------------

        plt.title(f"{feat.upper()} DISTRIBUTION", fontsize=14, pad=20, fontweight='bold')
        plt.legend(title=feat, bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False)
        plt.ylabel("Percentage (%)")
        plt.xlabel("Cluster Number")
        plt.xticks(rotation=0)

        plt.tight_layout()
        plt.show()

# Exemplo de uso:
plot_stacked_bars(df1, ['gender', 'time_period'], 'hc_merged_labels')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_grouped_bars(df, features, cluster_col):
    # Defining a custom pastel color palette

    custom_pastel = ['#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#F9F7CF']

    for feat in features:
        # Set the white style before creating the figure
        sns.set_style("white")
        plt.figure(figsize=(12, 6), facecolor='white')

        # Create grouped bar chart
        ax = sns.countplot(
            data=df,
            x=cluster_col,
            hue=feat,
            palette=custom_pastel
        )

        # Add value labels on top of the bars
        for p in ax.patches:
            if p.get_height() > 0:
                ax.annotate(f'{int(p.get_height())}',
                            (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha = 'center', va = 'center',
                            xytext = (0, 9),
                            textcoords = 'offset points',
                            fontsize=10,
                            fontweight='bold',
                            color='#555555')

        # --- AESTHETIC ADJUSTMENTS ---
        plt.title(f"{feat.upper()} COUNT", fontsize=15, pad=20, fontweight='bold')
        plt.xlabel("Cluster Number", fontsize=12)
        plt.ylabel("Quantity", fontsize=12)

        # Legend adjustment (white background and no border)
        plt.legend(title=feat, bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False)

        # Remove unnecessary spines (top and right by default)
        sns.despine()

        # Ensure the axis background is also white
        ax.set_facecolor('white')

        plt.tight_layout()
        plt.show()

# Function call
plot_grouped_bars(df1, ['gender', 'time_period'], 'hc_merged_labels')

### Cluster Visualization using TSNE

In [ ]:
def plot_tsne(df, feats, label,
              cmap='tab10',
              title="t-SNE Visualization of Ward Clustering Solution"):

  two_dim = TSNE(random_state=42).fit_transform(df[feats])
  two_dim_df = pd.DataFrame(two_dim, index=df.index)
  two_dim_df[label] = df[label]


  fig, ax= plt.subplots(figsize=(10,10))
  scatter = ax.scatter(x = two_dim_df[0],
                      y=two_dim_df[1],
                      c=two_dim_df[label],
                      s=5,
                      cmap=cmap
                      )
  ax.set_xlabel("")
  ax.set_ylabel("")
  ax.set_xticks([])
  ax.set_yticks([])

  legend1 = ax.legend(*scatter.legend_elements(),
                      loc="best", title="Cluster Labels")
  ax.add_artist(legend1)

  plt.title(title)
  plt.show()

In [ ]:
plot_tsne(df1, numerical, merged_label_name)

###  Comparative Analysis: K-Means vs. Ward’s Method

The consistency of the identified segments is further validated by comparing **K-Means** with **Ward’s Hierarchical Method**. While both algorithms identify virtually identical general groups, the **t-SNE** visualization reveals that **Ward’s Method** produces significantly more **compact and well-separated clusters**. Ward’s algorithm better handles the nuances of the merged perspectives, minimizing overlap and highlighting the distinct boundaries of the four final segments.


### Decision Tree

In [ ]:
# Preparing the data
X = df1[numerical]
y = df1[merged_label_name]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fitting the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=4)
dt.fit(X_train, y_train)
print("It is estimated that on average, we are able to predict {0:.2f}% of the customers correctly".format(dt.score(X_test, y_test)*100))


In [ ]:
# Assessing feature importance
pd.Series(dt.feature_importances_, index=X_train.columns).sort_values(ascending=False)


In [ ]:
## Predicting the cluster labels of the outliers

df_noise[merged_label_name] = dt.predict(df_noise[numerical])
df_noise[numerical + [merged_label_name]].head()

In [ ]:
# Visualizing the decision tree
num_of_clusters = len(df1[merged_label_name].unique())
cluster_names = ["Cluster {}".format(i) for i in range(num_of_clusters)]
dot_data = export_graphviz(dt, out_file=None,
                           feature_names=X.columns.to_list(),
                           filled=True,
                           rounded=True,
                           class_names=cluster_names,
                           special_characters=True)


In [ ]:
### optionally save dt png
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('cluster_decision_tree2.png')

## Open sidebar to download the image

# Profilling

In [ ]:
# Profilling each cluster (only merged)
merged_label_list = [merged_label_name]
sns.set(style="white")
cluster_profiles(
    df1 = df1[numerical + merged_label_list],
    label_columns = merged_label_list,
    figsize = (27, 8),
    compar_titles = ["Merged clusters profiling"],
    colors='Set2'
)


In [ ]:
cluster_means = get_mean_bylabel(df_unscaled,
                                 numerical,
                                 merged_label_name)

cluster_means.style.format(precision=2).background_gradient(axis=0)


### **Final Cluster Profiling: Merged Perspectives**

Based on the integration of demographic and preference data, the following four segments have been identified as the definitive customer profiles:

#### **Cluster 0: The Price-Sensitive & Fussy Shoppers**

* **Behavioural Profile**: This group is behaviourally distinct due to high interaction levels; they have the highest average **complaints** (**1.29**) and visit the most physical **stores** (**2.81**). They are "Cherry Pickers" who rely heavily on **promotions** (**0.64** average `pct_promo`).
* **Spending Habits**: Their budget is strictly managed, showing the lowest expenditure on **groceries** (**9,683.17**) and low interest in high-ticket categories.
* **Demographic Context**: Average age of **55.7** with a moderate number of children (**1.51**).
* **Strategic Insight**: To capture value here, the business should use deep-discount loss-leaders to drive foot traffic, as these shoppers are highly mobile and willing to switch stores for a better price.

#### **Cluster 1: The Niche Entertainment Enthusiasts**

* **Behavioural Profile**: This group is defined by extreme spending in leisure categories. They lead the dataset in **Video Games** (**5,005.34**) and **Alcohol** (**2,565.19**).
* **Spending Habits**: They show significantly lower interest in household staples like fresh produce, focusing their disposable income on entertainment and electronics.
* **Demographic Context**: They represent the "Younger" profile with the fewest children (**0.40**) and a preference for late-night shopping (**typical_hour: 17.81**).
* **Strategic Insight**: This is a "leisure" shopper. Marketing should focus on high-end tech launches, gaming accessories, and premium beverage selections.

#### **Cluster 2: The Premium Family Pillar**

* **Size**: ~6,800 observations (the largest and most profitable segment).
* **Behavioural Profile**: This cluster is the "engine" of the store, dominating spending across all core fresh categories: **Groceries** (**45,007.24**), **Fish** (**2,785.22**), and **Meat** (log: **8.23**).
* **Demographic Context**: They perfectly align with a "Traditional Family" profile, having the highest child count (**4.35**) and high overall loyalty.
* **Strategic Insight**: Loyalty is key here. Strategies should include family-sized packaging, high-quality fresh product guarantees, and multi-buy offers on household staples.

#### **Cluster 3: The Health-Conscious Minimalists**

* **Behavioural Profile**: Defined by a disciplined and specific diet. They show the highest focus on **Vegetables** (**4,924.48**) while significantly reducing expenditure on **Meat** (log: **5.96**).
* **Spending Habits**: Their basket is "clean" and focused on natural, fresh produce rather than processed goods or non-essentials.
* **Demographic Context**: Stable, middle-aged shoppers (avg age **55.9**) with a moderate family size (**2.39** children).
* **Strategic Insight**: This group is motivated by health. Cross-selling organic products, plant-based meat alternatives, and eco-friendly household goods would be highly effective.



In [ ]:

def plot_stacked_bars(df, features, cluster_col):
    # Defining a custom pastel palette
    pastel_colors = ['#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#F9F7CF']

    for feat in features:
        cross_tab = pd.crosstab(df[cluster_col], df[feat], normalize='index') * 100

        # Creating the figure and defining the white background
        fig, ax = plt.subplots(figsize=(10, 6), facecolor='white')

        cross_tab.plot(kind='bar', stacked=True, ax=ax, color=pastel_colors, edgecolor='white')

        # Add the percentage labels
        for p in ax.patches:
            width, height = p.get_width(), p.get_height()
            if height > 5:
                ax.annotate(f'{height:.1f}%', (p.get_x() + width/2, p.get_y() + height/2),
                            ha='center', va='center', fontsize=9, fontweight='bold', color='#444444')

        # --- BACKGROUND AND AESTHETIC ADJUSTMENTS ---
        ax.set_facecolor('white')           # Background of the white chart area
        ax.grid(False)                      # remove the grid lines
        ax.spines['top'].set_visible(False) # Remove top border
        ax.spines['right'].set_visible(False) # Remove right border
        # ----------------------------------

        plt.title(f"{feat.upper()} DISTRIBUTION", fontsize=14, pad=20, fontweight='bold')
        plt.legend(title=feat, bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False)
        plt.ylabel("Percentage (%)")
        plt.xlabel("Cluster Number")
        plt.xticks(rotation=0)

        plt.tight_layout()
        plt.show()

# Exemplo de uso:
plot_stacked_bars(df1, ['gender', 'time_period'], 'hc_merged_labels')

The integration of temporal data and gender demographics provides some context for the final segments:

**Temporal Patterns**: Shopping schedules vary significantly by lifestyle. Cluster 1 (Gamers) is the most nocturnal group, conducting 60.6% of their purchases in the evening. Cluster 2 (Families) maintains a more traditional routine with the highest morning activity at 38.7%. Clusters 0 (Deal hunters) and 3 (Vegans) predominantly shop during the afternoon.

**Gender Distribution:** The data reveals a near-perfect balance across all segments, with a consistent 50/50 split between male and female customers. This indicates that consumption drivers, such as the high-tech focus in Cluster 1 or the health focus in Cluster 3, are independent of gender.